In [2]:
import numpy as np
import pandas as pd
import json

import konlpy
from konlpy.tag import Okt

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flower_recommendation import get_user_input_vector # 우리 모델 

## 모델 성능 평가
**정답지 구성 방안 / 사용자 멘트 내용들은 고려되지 않기에 무조건 100% 맞다고 볼 수 없음. 참고용으로 사용**
1. 시맨틱 키워드 추출
2. 정답지(사용자 입력)에 시맨틱 키워드 추출
3. 키워드 포함된 답안지 구축
4. 코사인 유사도를 통해 정답 꽃 생성 -> 정답지 구성

**bert모델과 정확도 비교**

In [20]:
#파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋7(월,일,색상,RGB).csv"
correct_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/추천시스템_데이터_평가.csv"

In [26]:
df = pd.read_csv(df_file) #, encoding='CP949'
df.drop(["일자", "선택한 색상", "중앙 색상들"], axis=1, inplace=True)
df.head(5)

,꽃,월,계절,꽃말,설명,이미지,색상
0,각시붓꽃,3,봄,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,보라
1,감국,10,가을,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,노랑
2,개나리,3,봄,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",https://file.honestflower.kr/media/images/ingr...,노랑
3,개나리,4,봄,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",https://file.honestflower.kr/media/images/ingr...,노랑
4,갯개미취,9,가을,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,https://lh6.googleusercontent.com/proxy/yRZlN4...,보라


In [44]:
#필요한 칼럼만 가져오기
df_nlp = df[["꽃", "월", "계절", "꽃말", "설명", "색상"]]
print(df_nlp.shape)
df_nlp.head(5)

(837, 6)


,꽃,월,계절,꽃말,설명,색상
0,각시붓꽃,3,봄,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,보라
1,감국,10,가을,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,노랑
2,개나리,3,봄,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",노랑
3,개나리,4,봄,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",노랑
4,갯개미취,9,가을,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,보라


In [45]:
df_nlp["월"] = df_nlp["월"].astype(str)

C:\Users\pc\AppData\Local\Temp\ipykernel_8208\3142737305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp["월"] = df_nlp["월"].astype(str)


In [46]:
#월/꽃말/설명/색상 합치기
df_nlp['설명'] = df_nlp['월']+ "월. " + df_nlp['계절'] + ". " + df_nlp['꽃말'] + ". " + df_nlp['색상'] + ". " + df_nlp['설명']
df_nlp.drop(["월", "꽃말", "색상"], axis=1, inplace=True)
df_nlp.head(5)

C:\Users\pc\AppData\Local\Temp\ipykernel_8208\1584288816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['월']+ "월. " + df_nlp['계절'] + ". " + df_nlp['꽃말'] + ". " + df_nlp['색상'] + ". " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_8208\1584288816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp.drop(["월", "꽃말", "색상"], axis=1, inplace=True)


,꽃,계절,설명
0,각시붓꽃,봄,"3월. 봄. 부끄러움, 세련됨. 보라. 각시라 하면 이제 막 시집 온 새색시를 연상..."
1,감국,가을,10월. 가을. 그윽한 향기. 노랑. 가을 산야는 국화과 식물들 차지다. 특히 노란...
2,개나리,봄,"3월. 봄. 희망,깊은 정, 달성. 노랑. 우리나라 전역에서 봄 소식을 가장 먼저 ..."
3,개나리,봄,"4월. 봄. 희망. 노랑. 개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃..."
4,갯개미취,가을,9월. 가을. 추억. 보라. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 ...


In [47]:
#특수문자 및 띄어쓰기 제거
import re

def remove_special_characters(text):
    if not isinstance(text, str):
        text = str(text)   
        
    # 특수문자 제거
    pattern = r'[^\w\s\.]' #문자,공백문자,마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' #중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
df_nlp.head(10)

C:\Users\pc\AppData\Local\Temp\ipykernel_8208\379896977.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)


,꽃,계절,설명
0,각시붓꽃,봄,3월. 봄. 부끄러움 세련됨. 보라. 각시라 하면 이제 막 시집 온 새색시를 연상케...
1,감국,가을,10월. 가을. 그윽한 향기. 노랑. 가을 산야는 국화과 식물들 차지다. 특히 노란...
2,개나리,봄,3월. 봄. 희망깊은 정 달성. 노랑. 우리나라 전역에서 봄 소식을 가장 먼저 알려...
3,개나리,봄,4월. 봄. 희망. 노랑. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이...
4,갯개미취,가을,9월. 가을. 추억. 보라. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 ...
5,거베라,가을,10월. 가을. 신비 수수께끼. 분홍. 거베라만큼 크고 화려한 꽃도 드물다. 부를 ...
6,검은포플라,겨울,1월. 겨울. 용기. 빨강. 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습...
7,겹 공작초,가을,9월. 가을. 첫눈에 반함. 보라. 자기가 가장 아름답다고 믿는 세 여신이 있었다....
8,겹 과꽃,봄,4월. 봄. 나의 사랑은 당신보다깊다. 보라. 괴테의 희곡 파우스트 에 등장하는 소...
9,겹 소국,가을,9월. 가을. 청초. 분홍. 국화 품종 중 작은 화형을 가진 국화를 소국이라고 하며...


### 1. 시맨틱 키워드 추출
*정확한 단어만 파악가능하기 때문에 월, 계절, 색상만 사용

In [48]:
# 시맨틱 키워드 추출 (명확한 것들로 구성 / 월, 계절, 색상)
keywords = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월', '봄', '여름', '가을', '겨울', '빨강', '주황', '노랑', '초록', '파랑', '보라', '분홍', '갈색', '흰색']
flower_keywords = {flower: [keyword for keyword in keywords if keyword in description]
                   for flower, description in zip(df_nlp['꽃'], df_nlp['설명'])}

### 2. 정답지(사용자 입력)에 시맨틱 키워드 추출

In [49]:
#답안지 불러오기
correct = pd.read_csv(correct_file, encoding='CP949')
correct.head(5)

,Unnamed: 0,input,output
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"['감국', '구절초', '국화', '꽃무릇(석산)', '꽃아그배나무꽃', '꽃향유..."
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"['꽃 아로니아', '꽃 영춘화', '꽃아까시나무꽃', '처녀치마', '철쭉']"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"['거베라', '겹 소국', '겹 스토크', '겹 프릴 리시안셔스', '겹복숭아 꽃..."
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"['겹설유화', '공작초', '구즈마니아', '군자란', '꽃동백', '남천', '..."
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"['겹 알스트로메리아', '골담초', '까실쑥부쟁이', '깽깽이풀', '꽃동백', ..."


In [50]:
# 사용자 입력 텍스트(답안지 데이터)에서 시맨틱 키워드 추출
def extract_keywords(text):
    okt = Okt()
    tokens = okt.morphs(text)
    return [token for token in tokens if token in keywords]

correct['keyword'] = correct['input'].apply(extract_keywords)

### 3. 키워드 포함된 답안지 구축

In [51]:
# 각 input에 대해 keyword를 파싱하여 해당하는 꽃들을 찾아서 output 열에 추가하는 함수
correct['output2']= '0'

def find_matching_flowers(data):
    for index, row in data.iterrows():
        keyword = row['keyword']  # 문자열로 된 리스트를 파이썬 리스트로 변환
        matching_flowers = []
        
        for flower, properties in flower_keywords.items():
            if all(prop in properties for prop in keyword):
                matching_flowers.append(flower)
        data.at[index, 'output2'] = matching_flowers #원본 데이터프레임에 반영
    return data

In [52]:
# 결과 출력
correct = find_matching_flowers(correct)
correct.head(5)

,Unnamed: 0,input,output,keyword,output2
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"['감국', '구절초', '국화', '꽃무릇(석산)', '꽃아그배나무꽃', '꽃향유...",[가을],"[감국, 갯개미취, 거베라, 겹 공작초, 겹 소국, 겹 스토크, 겹헬레보러스, 계란..."
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"['꽃 아로니아', '꽃 영춘화', '꽃아까시나무꽃', '처녀치마', '철쭉']","[5월, 봄]","[겹벗꽃, 골든볼, 괭이밥, 귀고리꽃, 금낭화, 꽃 아로니아, 꽃 영춘화, 꽃아까시..."
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"['거베라', '겹 소국', '겹 스토크', '겹 프릴 리시안셔스', '겹복숭아 꽃...",[분홍],"[거베라, 겹 소국, 겹 스토크, 겹 프릴 리시안셔스, 겹복숭아 꽃, 골렘 카네이션..."
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"['겹설유화', '공작초', '구즈마니아', '군자란', '꽃동백', '남천', '...",[겨울],"[검은포플라, 겹 알스트로메리아, 겹설유화, 골담초, 공작초, 괭이눈, 구즈마니아,..."
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"['겹 알스트로메리아', '골담초', '까실쑥부쟁이', '깽깽이풀', '꽃동백', ...",[2월],"[겹 알스트로메리아, 골담초, 까실쑥부쟁이, 깽깽이풀, 꽃동백, 남천, 납매, 냉이..."


In [98]:
# #정답지 저장
# correct[['input', 'output']].to_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/추천시스템_데이터_평가.csv")

### 4. 기존 모델과 정확도 비교
- Precision@K: Top-K 아이템들 중 실제로 사용자가 선호하는 아이템의 비율

## bert 모델 / klue모델과 정확도 비교

In [15]:
correct = correct[['input', 'output']]
correct.head(5)

,input,output
0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"[감국, 구절초, 국화, 꽃무릇(석산), 꽃아그배나무꽃, 꽃향유, 꽈리, 남천, 눈..."
1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"[꽃 아로니아, 꽃 영춘화, 꽃아까시나무꽃, 처녀치마, 철쭉]"
2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"[거베라, 겹 소국, 겹 스토크, 겹 프릴 리시안셔스, 겹복숭아 꽃, 골렘 카네이션..."
3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"[겹설유화, 공작초, 구즈마니아, 군자란, 꽃동백, 남천, 노루발풀, 눈개쑥부쟁이,..."
4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"[겹 알스트로메리아, 골담초, 까실쑥부쟁이, 깽깽이풀, 꽃동백, 남천, 납매, 냉이..."


In [4]:
#답안지 불러오기
correct_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/추천시스템_데이터_평가.csv"
correct = pd.read_csv(correct_file, encoding='cp949')
correct.drop('Unnamed: 0', axis=1, inplace=True)
correct.head(5)

,input,output
0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"['감국', '구절초', '국화', '꽃무릇(석산)', '꽃아그배나무꽃', '꽃향유..."
1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"['꽃 아로니아', '꽃 영춘화', '꽃아까시나무꽃', '처녀치마', '철쭉']"
2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"['거베라', '겹 소국', '겹 스토크', '겹 프릴 리시안셔스', '겹복숭아 꽃..."
3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"['겹설유화', '공작초', '구즈마니아', '군자란', '꽃동백', '남천', '..."
4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"['겹 알스트로메리아', '골담초', '까실쑥부쟁이', '깽깽이풀', '꽃동백', ..."


In [12]:
df_nlp= pd.read_csv("C:/Users/pc/Model/Model-9/dataset/추천시스템_데이터.csv")

# JSON 문자열을 배열로 변환
df_nlp['최종_벡터'] = df_nlp['최종_벡터'].apply(lambda x: np.array(json.loads(x)))
df_nlp.head(5)

,꽃,월,계절,꽃말,설명,설명2,설명_벡터,원핫인코딩,최종_벡터
0,각시붓꽃,3,봄,"부끄러움, 세련됨",부끄러움 세련됨. 보라. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래...,[CLS] 부끄러움 세련됨 [SEP] [CLS] 보라 [SEP] [CLS] 각시라 ...,"[[0.04113124683499336, 0.11827859282493591, -0...",[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1....,"[[0.04113124683499336, 0.11827859282493591, -0..."
1,감국,10,가을,그윽한 향기,그윽한 향기. 노랑. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지...,[CLS] 그윽한 향기 [SEP] [CLS] 노랑 [SEP] [CLS] 가을 산야는...,"[[0.06626041233539581, 0.14091618359088898, -0...",[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0....,"[[0.06626041233539581, 0.14091618359088898, -0..."
2,개나리,3,봄,"희망,깊은 정, 달성",희망깊은 정 달성. 노랑. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인...,[CLS] 희망깊은 정 달성 [SEP] [CLS] 노랑 [SEP] [CLS] 우리나...,"[[0.12369333207607269, 0.1469401866197586, -0....",[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1....,"[[0.12369333207607269, 0.1469401866197586, -0...."
3,개나리,4,봄,희망,희망. 노랑. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리...,[CLS] 희망 [SEP] [CLS] 노랑 [SEP] [CLS] 개나리 봄을 알리는...,"[[0.07213923335075378, 0.13994745910167694, -0...",[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1....,"[[0.07213923335075378, 0.13994745910167694, -0..."
4,갯개미취,9,가을,추억,추억. 보라. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을...,[CLS] 추억 [SEP] [CLS] 보라 [SEP] [CLS] 옛날 일 따위는 깨...,"[[0.23332802951335907, 0.04906110465526581, -0...",[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0....,"[[0.23332802951335907, 0.04906110465526581, -0..."


In [19]:
# 추천 시스템 함수 (코사인 유사도 기반)
def recommend_flower(user_input, df):

    #코사인 유사도 산출
    df['유사도'] = df['최종_벡터'].apply(lambda x: cosine_similarity(user_input, x)[0][0])
    df['유사도'] = df['유사도'].astype(float) #숫자형으로 변환

    # 유사도를 기준으로 상위 3개의 꽃을 선택하고 중복된 꽃을 제거
    top3 = df.nlargest(3, '유사도').drop_duplicates(subset='꽃')

    # 만약 중복 제거 후 3개의 꽃이 되지 않는 경우, 다시 nlargest로 채우기
    if top3.shape[0] < 3:
        additional_top = df.nlargest(20, '유사도')  # 상위 10개 정도를 선택
        additional_top = additional_top[~additional_top['꽃'].isin(top3['꽃'])]
        top3 = pd.concat([top3, additional_top]).nlargest(3, '유사도').drop_duplicates(subset='꽃')

    return top3['꽃'].values.tolist()

In [20]:
#임의의 값 넣기
correct['bert_output'] = 0

In [21]:
# correct 데이터셋의 각 행에 대해 꽃 추천 진행
for index, row in correct.iterrows():
    bert_vector = row['bert_vector']
    correct['bert_output'][index] = recommend_flower(bert_vector, df_nlp)

C:\Users\pc\AppData\Local\Temp\ipykernel_24156\88125218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correct['bert_output'][index] = recommend_flower(bert_vector, df_nlp)


In [22]:
correct2 = correct[["input", "output", "bert_output"]]
correct2

,input,output,bert_output
0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"[감국, 구절초, 국화, 꽃무릇(석산), 꽃아그배나무꽃, 꽃향유, 꽈리, 남천, 눈...","[카푸치노 장미, 마거리트, 박하]"
1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"[꽃 아로니아, 꽃 영춘화, 꽃아까시나무꽃, 처녀치마, 철쭉]","[여주 카라, 옥시페탈룸, 아란다]"
2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"[거베라, 겹 소국, 겹 스토크, 겹 프릴 리시안셔스, 겹복숭아 꽃, 골렘 카네이션...","[프리지아, 딥퍼플 장미, 안개꽃]"
3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"[겹설유화, 공작초, 구즈마니아, 군자란, 꽃동백, 남천, 노루발풀, 눈개쑥부쟁이,...","[살구꽃, 장미, 하이앤보니타 장미]"
4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"[겹 알스트로메리아, 골담초, 까실쑥부쟁이, 깽깽이풀, 꽃동백, 남천, 납매, 냉이...","[펠리시티 장미, 살구꽃, 노랑 히야신]"
...,...,...,...
95,5월 봄 결혼식 축하 화환으로 어떤 꽃이 좋을까요?,"[꽃 아로니아, 꽃 영춘화, 꽃아까시나무꽃, 처녀치마, 철쭉]","[루치아노 장미, 시레네, 덴파레]"
96,오래된 연인에게 특별한 날에 봄에 기념할 꽃은 무엇이 있니?,"[개나리, 겹 수선화, 겹복숭아 꽃, 겹설유화, 공작초, 깽깽이풀, 꽃 아로니아, ...","[꽃 영춘화, 빨간아네모네, 조팝나무꽃]"
97,직장 동료에게 감사 편지와 함께 보낼 가을의 꽃을 추천해줘.,"[감국, 구절초, 국화, 꽃무릇(석산), 꽃아그배나무꽃, 꽃향유, 꽈리, 남천, 눈...","[스타티스, 코랄리프 장미, 공작초]"
98,공부에 지친 친구에게 10월에 줄 꽃을 추천받고 싶어요.,"[감국, 거베라, 겹 스토크, 괭이밥(옥살리스), 구절초, 금잔화, 꽃아그배나무꽃,...","[마거리트, 프리지아, 빨간국화]"


In [5]:
#Precision@K를 계산
def precision_at_k(actual, predicted, k=3):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(k) #교집합의 개수 / 추천 수
    return result

In [60]:
# 각 질문에 대한 Precision@3 계산
precisions = []
for index, row in correct2.iterrows():
    p_at_3 = precision_at_k(row["output"], row["bert_output"], k=3)
    precisions.append(p_at_3)
    # print(f"{row['input']} - Precision@3: {p_at_3:.3f}")

# 전체 평균 Precision@3 계산
avg_precision = sum(precisions) / len(precisions)
print(f"\n평균 Precision@3: {avg_precision:.3f}")


평균 Precision@3: 0.287


In [26]:
#klue 비교
df_klue = pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/1_klue+가중치(월포함.csv")
df_klue

,Unnamed: 0,user_input,추천꽃1,추천꽃2,추천꽃3
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,국화,수크령,구절초
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,데이지,카네이션,아네모네
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,인게이지먼트 장미,부용,무궁화
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,안스리움,마가렛,아네모네
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,스노우드롭,살구꽃,협죽도
...,...,...,...,...,...
95,95,5월 봄 결혼식 축하 화환으로 어떤 꽃이 좋을까요?,데이지,카네이션,덴파레
96,96,오래된 연인에게 특별한 날에 봄에 기념할 꽃은 무엇이 있니?,데이지,마가렛,물망초
97,97,직장 동료에게 감사 편지와 함께 보낼 가을의 꽃을 추천해줘.,수크령,부용,마가렛
98,98,공부에 지친 친구에게 10월에 줄 꽃을 추천받고 싶어요.,마가렛,천일홍,데이지


In [27]:
df_klue['추천꽃'] = "0"

In [28]:
for idx, row in df_klue.iterrows():
    flower_list=[]
    flower_list.append(row['추천꽃1'])
    flower_list.append(row['추천꽃2'])
    flower_list.append(row['추천꽃3'])
    df_klue.at[idx, '추천꽃'] = flower_list

In [33]:
df_klue.head(5)

,Unnamed: 0,user_input,추천꽃1,추천꽃2,추천꽃3,추천꽃
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,국화,수크령,구절초,"[국화, 수크령, 구절초]"
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,데이지,카네이션,아네모네,"[데이지, 카네이션, 아네모네]"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,인게이지먼트 장미,부용,무궁화,"[인게이지먼트 장미, 부용, 무궁화]"
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,안스리움,마가렛,아네모네,"[안스리움, 마가렛, 아네모네]"
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,스노우드롭,살구꽃,협죽도,"[스노우드롭, 살구꽃, 협죽도]"


In [35]:
correct3 = correct.copy()
correct3['klue_output'] = df_klue['추천꽃']
correct3.head(5)

,input,output,klue_output
0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"[감국, 구절초, 국화, 꽃무릇(석산), 꽃아그배나무꽃, 꽃향유, 꽈리, 남천, 눈...","[국화, 수크령, 구절초]"
1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"[꽃 아로니아, 꽃 영춘화, 꽃아까시나무꽃, 처녀치마, 철쭉]","[데이지, 카네이션, 아네모네]"
2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"[거베라, 겹 소국, 겹 스토크, 겹 프릴 리시안셔스, 겹복숭아 꽃, 골렘 카네이션...","[인게이지먼트 장미, 부용, 무궁화]"
3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"[겹설유화, 공작초, 구즈마니아, 군자란, 꽃동백, 남천, 노루발풀, 눈개쑥부쟁이,...","[안스리움, 마가렛, 아네모네]"
4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"[겹 알스트로메리아, 골담초, 까실쑥부쟁이, 깽깽이풀, 꽃동백, 남천, 납매, 냉이...","[스노우드롭, 살구꽃, 협죽도]"


In [38]:
# 각 질문에 대한 Precision@3 계산
precisions = []
for index, row in correct3.iterrows():
    p_at_3 = precision_at_k(row["output"], row["klue_output"], k=3)
    precisions.append(p_at_3)
    # print(f"{row['input']} - Precision@3: {p_at_3:.3f}")

# 전체 평균 Precision@3 계산
avg_precision = sum(precisions) / len(precisions)
print(f"\n평균 Precision@3: {avg_precision:.3f}")


평균 Precision@3: 0.453


In [65]:
#klue(수정) 비교
df_klue2 = pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/klue_테스트_모델결과(수정 후).csv")
df_klue2

,Unnamed: 0,user_input,추천꽃1,추천꽃2,추천꽃3
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,과꽃,수크령,매화
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,무릇,제비고깔,겹벗꽃
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,부용,인게이지먼트 장미,큐티버블 소국
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,인챈트먼트 장미,장미,풀몬티 장미
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,아도니스,네모필라,살구꽃
...,...,...,...,...,...
95,95,5월 봄 결혼식 축하 화환으로 어떤 꽃이 좋을까요?,카네이션,덴파레,데이지
96,96,오래된 연인에게 특별한 날에 봄에 기념할 꽃은 무엇이 있니?,보라색라일락,아도니스,라일락
97,97,직장 동료에게 감사 편지와 함께 보낼 가을의 꽃을 추천해줘.,수크령,메리골드,흰독말풀
98,98,공부에 지친 친구에게 10월에 줄 꽃을 추천받고 싶어요.,이끼장미,매화,흰독말풀


In [66]:
df_klue2['추천꽃'] = "0"

for idx, row in df_klue2.iterrows():
    flower_list=[]
    flower_list.append(row['추천꽃1'])
    flower_list.append(row['추천꽃2'])
    flower_list.append(row['추천꽃3'])
    df_klue2.at[idx, '추천꽃'] = flower_list

df_klue2.head(5)

,Unnamed: 0,user_input,추천꽃1,추천꽃2,추천꽃3,추천꽃
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,과꽃,수크령,매화,"[과꽃, 수크령, 매화]"
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,무릇,제비고깔,겹벗꽃,"[무릇, 제비고깔, 겹벗꽃]"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,부용,인게이지먼트 장미,큐티버블 소국,"[부용, 인게이지먼트 장미, 큐티버블 소국]"
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,인챈트먼트 장미,장미,풀몬티 장미,"[인챈트먼트 장미, 장미, 풀몬티 장미]"
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,아도니스,네모필라,살구꽃,"[아도니스, 네모필라, 살구꽃]"


In [67]:
correct4 = correct.copy()
correct4['klue_output2'] = df_klue2['추천꽃']
correct4.head(5)

,Unnamed: 0,input,output,keyword,output2,klue_output2
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"['감국', '구절초', '국화', '꽃무릇(석산)', '꽃아그배나무꽃', '꽃향유...",[가을],"[감국, 갯개미취, 거베라, 겹 공작초, 겹 소국, 겹 스토크, 겹헬레보러스, 계란...","[과꽃, 수크령, 매화]"
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"['꽃 아로니아', '꽃 영춘화', '꽃아까시나무꽃', '처녀치마', '철쭉']","[5월, 봄]","[겹벗꽃, 골든볼, 괭이밥, 귀고리꽃, 금낭화, 꽃 아로니아, 꽃 영춘화, 꽃아까시...","[무릇, 제비고깔, 겹벗꽃]"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"['거베라', '겹 소국', '겹 스토크', '겹 프릴 리시안셔스', '겹복숭아 꽃...",[분홍],"[거베라, 겹 소국, 겹 스토크, 겹 프릴 리시안셔스, 겹복숭아 꽃, 골렘 카네이션...","[부용, 인게이지먼트 장미, 큐티버블 소국]"
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"['겹설유화', '공작초', '구즈마니아', '군자란', '꽃동백', '남천', '...",[겨울],"[검은포플라, 겹 알스트로메리아, 겹설유화, 골담초, 공작초, 괭이눈, 구즈마니아,...","[인챈트먼트 장미, 장미, 풀몬티 장미]"
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"['겹 알스트로메리아', '골담초', '까실쑥부쟁이', '깽깽이풀', '꽃동백', ...",[2월],"[겹 알스트로메리아, 골담초, 까실쑥부쟁이, 깽깽이풀, 꽃동백, 남천, 납매, 냉이...","[아도니스, 네모필라, 살구꽃]"


In [68]:
import ast 

def convert_to_list(s):
    # '[' 부터 ']' 까지의 내용을 추출
    start = s.index('[')
    end = s.rindex(']')
    list_str = s[start:end+1]
    
    # 문자열을 리스트로 변환
    try:
        return ast.literal_eval(list_str)
    except:
        # 리스트 형식이 올바르지 않은 경우, 쉼표로 분리
        return [item.strip() for item in list_str[1:-1].split(',')]

In [33]:
correct4['output'] = correct4['output'].apply(convert_to_list)

In [69]:
#Precision@K를 계산
def precision_at_k(actual, predicted, k=3):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(k) #교집합의 개수 / 추천 수
    return result

In [70]:
# 각 질문에 대한 Precision@3 계산
precisions = []
for index, row in correct4.iterrows():
    p_at_3 = precision_at_k(row["output2"], row["klue_output2"], k=3)
    precisions.append(p_at_3)
    # print(f"{row['input']} - Precision@3: {p_at_3:.3f}")

# 전체 평균 Precision@3 계산
avg_precision = sum(precisions) / len(precisions)
print(f"\n평균 Precision@3: {avg_precision:.3f}")


평균 Precision@3: 0.773


In [247]:
# correct2.to_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/추천시스템_데이터_평가.csv")

In [72]:
#klue+tfidf 비교
df_klue_idf = pd.read_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/klue+tfidf_테스트_모델결과.csv") #, encoding='cp949'
df_klue_idf

,Unnamed: 0,user_input,추천꽃1,추천꽃2,추천꽃3
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,카네이션,알스트로메리아,수크령
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,무릇,데이지,제비고깔
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,금낭화,부용,인게이지먼트 장미
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,장미,꽃동백,뉴페이스 장미
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,살구꽃,복수초,서향
...,...,...,...,...,...
95,95,5월 봄 결혼식 축하 화환으로 어떤 꽃이 좋을까요?,덴파레,금잔화,작약
96,96,오래된 연인에게 특별한 날에 봄에 기념할 꽃은 무엇이 있니?,미나리아재비,빙카,보라색라일락
97,97,직장 동료에게 감사 편지와 함께 보낼 가을의 꽃을 추천해줘.,수크령,부용,메리골드
98,98,공부에 지친 친구에게 10월에 줄 꽃을 추천받고 싶어요.,이끼장미,매화,천일홍


In [73]:
df_klue_idf['추천꽃'] = "0"

for idx, row in df_klue_idf.iterrows():
    flower_list=[]
    flower_list.append(row['추천꽃1'])
    flower_list.append(row['추천꽃2'])
    flower_list.append(row['추천꽃3'])
    df_klue_idf.at[idx, '추천꽃'] = flower_list

df_klue_idf.head(5)

,Unnamed: 0,user_input,추천꽃1,추천꽃2,추천꽃3,추천꽃
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,카네이션,알스트로메리아,수크령,"[카네이션, 알스트로메리아, 수크령]"
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,무릇,데이지,제비고깔,"[무릇, 데이지, 제비고깔]"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,금낭화,부용,인게이지먼트 장미,"[금낭화, 부용, 인게이지먼트 장미]"
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,장미,꽃동백,뉴페이스 장미,"[장미, 꽃동백, 뉴페이스 장미]"
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,살구꽃,복수초,서향,"[살구꽃, 복수초, 서향]"


In [74]:
correct5 = correct.copy()
correct5['klue_idf_output'] = df_klue_idf['추천꽃']
correct5.head(5)

,Unnamed: 0,input,output,keyword,output2,klue_idf_output
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"['감국', '구절초', '국화', '꽃무릇(석산)', '꽃아그배나무꽃', '꽃향유...",[가을],"[감국, 갯개미취, 거베라, 겹 공작초, 겹 소국, 겹 스토크, 겹헬레보러스, 계란...","[카네이션, 알스트로메리아, 수크령]"
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"['꽃 아로니아', '꽃 영춘화', '꽃아까시나무꽃', '처녀치마', '철쭉']","[5월, 봄]","[겹벗꽃, 골든볼, 괭이밥, 귀고리꽃, 금낭화, 꽃 아로니아, 꽃 영춘화, 꽃아까시...","[무릇, 데이지, 제비고깔]"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"['거베라', '겹 소국', '겹 스토크', '겹 프릴 리시안셔스', '겹복숭아 꽃...",[분홍],"[거베라, 겹 소국, 겹 스토크, 겹 프릴 리시안셔스, 겹복숭아 꽃, 골렘 카네이션...","[금낭화, 부용, 인게이지먼트 장미]"
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"['겹설유화', '공작초', '구즈마니아', '군자란', '꽃동백', '남천', '...",[겨울],"[검은포플라, 겹 알스트로메리아, 겹설유화, 골담초, 공작초, 괭이눈, 구즈마니아,...","[장미, 꽃동백, 뉴페이스 장미]"
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"['겹 알스트로메리아', '골담초', '까실쑥부쟁이', '깽깽이풀', '꽃동백', ...",[2월],"[겹 알스트로메리아, 골담초, 까실쑥부쟁이, 깽깽이풀, 꽃동백, 남천, 납매, 냉이...","[살구꽃, 복수초, 서향]"


In [75]:
import ast 

def convert_to_list(s):
    # '[' 부터 ']' 까지의 내용을 추출
    start = s.index('[')
    end = s.rindex(']')
    list_str = s[start:end+1]
    
    # 문자열을 리스트로 변환
    try:
        return ast.literal_eval(list_str)
    except:
        # 리스트 형식이 올바르지 않은 경우, 쉼표로 분리
        return [item.strip() for item in list_str[1:-1].split(',')]

In [76]:
#Precision@K를 계산
def precision_at_k(actual, predicted, k=3):
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(k) #교집합의 개수 / 추천 수
    return result

In [77]:
# 각 질문에 대한 Precision@3 계산
precisions = []
for index, row in correct5.iterrows():
    p_at_3 = precision_at_k(row["output2"], row["klue_idf_output"], k=3)
    precisions.append(p_at_3)
    # print(f"{row['input']} - Precision@3: {p_at_3:.3f}")

# 전체 평균 Precision@3 계산
avg_precision = sum(precisions) / len(precisions)
print(f"\n평균 Precision@3: {avg_precision:.3f}")


평균 Precision@3: 0.777
